In [369]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
from urllib.parse import quote

In [370]:
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

In [371]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
book_list = soup.select('.sub_book_list')
len(book_list)

20

In [372]:
book=book_list[1]
book

<li class="sub_book_list">
<div class="view_box">
<!--
            <span class="view_box_block" onclick="location.href='/store/books/look.php?p_code=B6972407557';">
            -->
<span class="view_box_block">
<span class="img_hidden_b">
<span class="pop_quick_menu">
<a href="/store/books/look.php?p_code=B6972407557"><img alt="상세보기" src="https://www.hanbit.co.kr/images/common/icon_view_detail.png"/></a>
</span>
<span class="price">14,400<span>원</span></span>
<span class="pop_quick_bg" onclick="location='/store/books/look.php?p_code=B6972407557'"></span>
</span>
<!-- 예약도서인 경우 -->
<!-- ebook이 도서인 경우 -->
<img alt="" class="thumb" src="/data/books/B6972407557_m.jpg"/>
</span>
<div class="info">
<p class="book_tit"><a href="/store/books/look.php?p_code=B6972407557">다시, 배우다</a></p>
<p class="book_writer">폴 김 </p>
</div>
</div>
</li>

In [373]:
name=book.select_one('.book_tit').get_text()
name

'다시, 배우다'

In [374]:
author=book.select_one('.book_writer').get_text()
author

'폴 김 '

In [375]:
url='https://www.hanbit.co.kr'

In [376]:
href=book_list[1].find('a')['href']
href
sub_url = url + href
sub_url

'https://www.hanbit.co.kr/store/books/look.php?p_code=B6972407557'

In [377]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [378]:
translator_1 = sub_soup.select_one('.info_list').get_text()
translator_1

'\n저자 : 폴 김 \n출간 : 2021-12-10\n페이지 : 304 쪽\nISBN : 9791157845545\n물류코드 :3356\n'

In [379]:
translator_1 = sub_soup.select_one('.info_list').get_text()
if '번역' in translator_1:
    translator = sub_soup.select_one('.info_list').get_text().split('\n')[2].split(':')[1]
else :
    translator = 'x'


In [380]:
publish_date = sub_soup.select_one('.info_list').get_text().split('\n')[3].split(': ')[1]
publish_date

'304 쪽'

In [381]:
pages = sub_soup.select_one('.info_list').get_text().split('\n')[4].split(': ')[1]
pages

'9791157845545'

In [382]:
old_price = sub_soup.select('span.pbr')[0].get_text().replace('원','')
old_price

'16,000'

In [383]:
dis_price = str(sub_soup.select('span.pbr>strong')[0]).replace('strong','').replace('<>','').replace('</>','')
dis_price

'14,400'

In [384]:
mileage = str(sub_soup.select('span.pbr')[2]).replace('<span class="pbr">','').replace('</span>','')
mileage

'800점 (5%)'

In [385]:
lines = []

for book_1 in book_list:
    name=book_1.select_one('.book_tit').get_text()
    author=book_1.select_one('.book_writer').get_text()
    
    href=book_1.find('a')['href']
    
    sub_url = url + href
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')

    translator_1 = sub_soup.select_one('.info_list').get_text()

    if  '번역' in translator_1:
        translator = sub_soup.select_one('.info_list').get_text().split('\n')[2].split(':')[1]
        publish_date = sub_soup.select_one('.info_list').get_text().split('\n')[3].split(': ')[1]
        pages = sub_soup.select_one('.info_list').get_text().split('\n')[4].split(': ')[1]
        old_price = sub_soup.select('span.pbr')[0].get_text().replace('원','')
        dis_price = str(sub_soup.select('span.pbr>strong')[0]).replace('strong','').replace('<>','').replace('</>','')
        mileage = str(sub_soup.select('span.pbr')[2]).replace('<span class="pbr">','').replace('</span>','')
    else :
        translator = 'x'
        publish_date = sub_soup.select_one('.info_list').get_text().split('\n')[2].split(': ')[1]
        pages = sub_soup.select_one('.info_list').get_text().split('\n')[3].split(': ')[1]
        old_price = sub_soup.select('span.pbr')[0].get_text().replace('원','')
        dis_price = str(sub_soup.select('span.pbr>strong')[0]).replace('strong','').replace('<>','').replace('</>','')
        mileage = str(sub_soup.select('span.pbr')[2]).replace('<span class="pbr">','').replace('</span>','')

    lines.append([name,author,translator,publish_date,pages,old_price,dis_price,mileage])

In [386]:
df = pd.DataFrame(lines, columns=[
    '책이름','저자', '역자', '출판일', '페이지수', '정가','할인가','적립마일리지'
])

df

,책이름,저자,역자,출판일,페이지수,정가,할인가,적립마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",2021-12-14,416 쪽,"19,800","17,820",990점 (5%)
1,"다시, 배우다",폴 김,x,2021-12-10,304 쪽,"16,000","14,400",800점 (5%)
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",2021-12-10,376 쪽,"22,000","19,800","1,100점 (5%)"
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",x,2021-12-01,384 쪽,"18,000","16,200",900점 (5%)
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",2021-11-30,600 쪽,"36,000","36,000","1,080점 (3%)"
5,엑셀 매크로&VBA 업무 공략집,최준선,x,2021-11-30,328 쪽,"22,000","19,800","1,100점 (5%)"
6,데이터 과학자 되는 법,"에밀리 로빈슨 , 재클린 놀리스",이창화,2021-11-26,372 쪽,"24,000","21,600","1,200점 (5%)"
7,돈 되는 기획,김도균,x,2021-11-26,240 쪽,"14,500","13,050",725점 (5%)
8,빠르게 찾아 바로 적용하는 엑셀 에센스 사전 100,한은숙,x,2021-11-25,300 쪽,"18,000","16,200",900점 (5%)
9,초등 어휘력이 공부력이다,박명선,x,2021-11-25,412 쪽,"18,000","16,200",900점 (5%)


In [387]:
lines = []
for page in range(1,11):
    url = 'https://www.hanbit.co.kr/store/books/new_book_list.html?page='+str(page)+'&brand=&cate1=&cate2=&searchKey=&keyWord='
    res = requests.get(url)
    soup=BeautifulSoup(res.text, 'html.parser')
    book_list = soup.select('.sub_book_list')



    for book_1 in book_list:
        name=book_1.select_one('.book_tit').get_text()
        author=book_1.select_one('.book_writer').get_text()
        
        href=book_1.find('a')['href']
        
        url_2 = 'https://www.hanbit.co.kr'
        sub_url = url_2 + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')

        translator_1 = sub_soup.select_one('.info_list').get_text()

        if  '번역' in translator_1:
            translator = sub_soup.select_one('.info_list').get_text().split('\n')[2].split(':')[1]
            publish_date = sub_soup.select_one('.info_list').get_text().split('\n')[3].split(': ')[1]
            pages = sub_soup.select_one('.info_list').get_text().split('\n')[4].split(': ')[1]
            old_price = sub_soup.select('span.pbr')[0].get_text().replace('원','')
            dis_price = str(sub_soup.select('span.pbr>strong')[0]).replace('strong','').replace('<>','').replace('</>','')
            mileage = str(sub_soup.select('span.pbr')[2]).replace('<span class="pbr">','').replace('</span>','')
        else :
            translator = 'x'
            publish_date = sub_soup.select_one('.info_list').get_text().split('\n')[2].split(': ')[1]
            pages = sub_soup.select_one('.info_list').get_text().split('\n')[3].split(': ')[1]
            old_price = sub_soup.select('span.pbr')[0].get_text().replace('원','')
            dis_price = str(sub_soup.select('span.pbr>strong')[0]).replace('strong','').replace('<>','').replace('</>','')
            mileage = str(sub_soup.select('span.pbr')[2]).replace('<span class="pbr">','').replace('</span>','')

        lines.append([name,author,translator,publish_date,pages,old_price,dis_price,mileage])

In [394]:
df2 = pd.DataFrame(lines, columns=[
    '책이름','저자', '역자', '출판일', '페이지수', '정가','할인가','적립마일리지'
])

df2.tail(150)

,책이름,저자,역자,출판일,페이지수,정가,할인가,적립마일리지
50,모바일 UX/UI 디자인 강의 with Adobe XD,김영삼,x,2021-08-30,404 쪽,"26,000","23,400","1,300점 (5%)"
51,인문학 거저보기 : 서양철학 편,지하늘,x,2021-08-30,380 쪽,"18,500","16,650",925점 (5%)
52,처음 배우는 플라스크 웹 프로그래밍,윤정현,x,2021-08-30,260 쪽,"19,800","17,820",990점 (5%)
53,"IT CookBook, 현대 센서공학(5판)",Jacob Fraden,"김윤 , 김혁 , 박동욱",2021-08-27,688 쪽,"35,000","35,000","1,050점 (3%)"
54,방구석 심리학 실험실,마이클 A. 브릿,류초롱,2021-08-25,388 쪽,"18,000","16,200",900점 (5%)
...,...,...,...,...,...,...,...,...
195,"IT CookBook, C 언어 일취월장(2판)",신윤환,x,2021-02-24,800 쪽,"28,000","28,000",840점 (3%)
196,소비자행동,김학윤,x,2021-02-23,660 쪽,"35,000","35,000","1,050점 (3%)"
197,재미있고 빠른 어휘력 동화 – 두근두근 이 마음은 뭘까,"김세실 , 김도윤(그림)",x,2021-02-20,104 쪽,"12,000","10,800",600점 (5%)
198,재미있고 빠른 어휘력 동화 – 키득키득 재밌는 의성어 의태어,"김예실 , 김희선(그림)",x,2021-02-20,104 쪽,"12,000","10,800",600점 (5%)


In [389]:
df2.to_csv('한빛출판사_new_book.csv', index=False)